# YLL Sensitivity Analysis

Plot food consumption (kcal/person/day) vs valued YLL (USD) on a logarithmic scale.
Shows how consumption patterns shift as the value placed on years of life lost increases.

In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
from sensitivity_utils import (
    aggregate_food_groups,
    assign_food_colors,
    extract_consumption_data,
    extract_ghg_data,
    extract_objective_data,
    extract_scenarios_with_param,
    get_log_ticks,
    load_food_to_group,
    plot_objective_sensitivity,
    plot_stacked_sensitivity,
    prepare_objective_data,
)

In [ ]:
# Configuration
CONFIG_NAME = "yll"
PROJECT_ROOT = Path("..").resolve()
RESULTS_DIR = PROJECT_ROOT / "results" / CONFIG_NAME
PROCESSING_DIR = PROJECT_ROOT / "processing" / CONFIG_NAME

# Load food to group mapping
FOOD_TO_GROUP = load_food_to_group(PROJECT_ROOT)

# Constants
CONSTANT_HEALTH_VALUE_PER_YLL = 10000
CONSTANT_GHG_PRICE = 100
N_WORKERS = 8

In [ ]:
# Extract scenarios from config
scenarios = extract_scenarios_with_param(
    PROJECT_ROOT,
    CONFIG_NAME,
    param_path=["health", "value_per_yll"],
    scenario_prefix="yll_",
)

# Filter to only include scenarios with existing network files
scenarios = [(p, s, f) for p, s, f in scenarios if f.exists()]

print(f"Found {len(scenarios)} YLL scenarios:")
for yll_val, name, _ in scenarios:
    print(f"  {name}: {yll_val:,.0f} USD/YLL")

# Extract parameter values for tick generation
param_values = [p for p, _, _ in scenarios]

## Food Consumption

In [ ]:
# Extract consumption data
df = extract_consumption_data(
    scenarios,
    FOOD_TO_GROUP,
    RESULTS_DIR / "plots" / "yll_sensitivity.csv",
    param_name="yll_value",
    n_workers=N_WORKERS,
)

print(f"\nConsumption data shape: {df.shape}")
df

In [ ]:
# Aggregate food groups and prepare for plotting
df_plot = aggregate_food_groups(df)

# Order groups by consumption in the lowest YLL scenario (descending) for consistent stacking
min_yll = df_plot.index.min()
group_order = df_plot.loc[min_yll].sort_values(ascending=False).index.tolist()
df_plot = df_plot[group_order]

# Assign colors
colors = assign_food_colors(df_plot)

print(
    f"Food groups (ordered by yll_{int(min_yll)} caloric consumption, bottom to top):"
)
for i, group in enumerate(group_order):
    print(f"  {i}: {group}: {df_plot.loc[min_yll, group]:.1f} kcal/person/day")

In [ ]:
# X-axis configuration - derive from scenarios
YLL_XTICKS, YLL_XTICKLABELS = get_log_ticks(param_values)
YLL_XLABEL = "Additional value per Year of Life Lost [USD/YLL]"

print(f"X-axis ticks: {YLL_XTICKS}")
print(f"X-axis labels: {YLL_XTICKLABELS}")

# Manual label positions
LABEL_X_POSITIONS = {
    "grain": 7,
    "dairy": 30,
    "starchy_vegetable": 5,
    "legumes": 3000,
    "oil": 5,
    "red_meat": 3,
    "sugar": 10,
    "nuts_seeds": 10000,
    "whole_grains": 1000,
    "fruits_vegetables": 300,
    "eggs_poultry": 30,
}

In [ ]:
fig, ax = plt.subplots(figsize=(3.54, 2.5))  # 90mm x ~63mm
plot_stacked_sensitivity(
    df_plot,
    colors,
    ax,
    xlabel=YLL_XLABEL,
    ylabel="Food consumption [kcal/person/day]",
    panel_label="a",
    x_ticks=YLL_XTICKS,
    x_ticklabels=YLL_XTICKLABELS,
    label_x_positions=LABEL_X_POSITIONS,
    y_max=2400,
)
plt.tight_layout()
plt.savefig(RESULTS_DIR / "plots" / "yll_sensitivity.pdf", dpi=300, bbox_inches="tight")
plt.show()

## Objective Breakdown

Plot objective components (billion USD) vs valued YLL on a logarithmic scale.
Health burden is valued at a constant $10,000/YLL for comparability across scenarios.

In [ ]:
# Extract objective data
df_obj = extract_objective_data(
    scenarios,
    RESULTS_DIR / "plots" / "yll_objective_breakdown.csv",
    param_name="yll_value",
    constant_health_value=CONSTANT_HEALTH_VALUE_PER_YLL,
    constant_ghg_price=CONSTANT_GHG_PRICE,
    n_workers=N_WORKERS,
)

print(f"\nObjective data shape: {df_obj.shape}")
df_obj

In [ ]:
# Prepare objective data
df_obj = prepare_objective_data(df_obj)

print("Objective categories (ordered):")
for cat in df_obj.columns:
    sign = "+" if df_obj[cat].mean() > 0 else "-"
    print(f"  {sign} {cat}: {df_obj[cat].mean():.1f} billion USD (mean)")

In [ ]:
# Manual label positions for objective
OBJ_LABEL_X_POSITIONS = {
    "Crop production": 50000,
    "Health burden": 10,
    "GHG cost": 3,
    "Trade": 100000,
    "Consumer values": 5000,
}

In [ ]:
fig, ax = plt.subplots(figsize=(3.54, 2.5))  # 90mm x ~63mm
plot_objective_sensitivity(
    df_obj,
    ax,
    xlabel=YLL_XLABEL,
    panel_label="c",
    x_ticks=YLL_XTICKS,
    x_ticklabels=YLL_XTICKLABELS,
    health_value=CONSTANT_HEALTH_VALUE_PER_YLL,
    ghg_price=CONSTANT_GHG_PRICE,
    label_x_positions=OBJ_LABEL_X_POSITIONS,
    highlight_cat="GHG cost",
)
plt.tight_layout()
plt.savefig(
    RESULTS_DIR / "plots" / "yll_objective_breakdown.pdf", dpi=300, bbox_inches="tight"
)
plt.show()

## GHG Emissions by Food Group

Plot GHG emissions (GtCO2eq) by food group vs valued YLL on a logarithmic scale.
Uses flow-based attribution to trace emissions from consumption back through trade and processing to production.

In [ ]:
# Extract GHG data
df_ghg = extract_ghg_data(
    scenarios,
    FOOD_TO_GROUP,
    RESULTS_DIR / "plots" / "yll_ghg_by_food_group.csv",
    param_name="yll_value",
    n_workers=N_WORKERS,
)

print(f"\nGHG data shape: {df_ghg.shape}")
min_yll_ghg = df_ghg.index.min()
print(
    f"Total GHG at yll_{int(min_yll_ghg)}: {df_ghg.loc[min_yll_ghg].sum():.2f} GtCO2eq"
)
df_ghg

In [ ]:
# Aggregate and use same order as panel a
df_ghg_plot = aggregate_food_groups(df_ghg)
df_ghg_plot = df_ghg_plot[group_order]

print("Food groups (same order as panel a, based on consumption):")
min_yll_ghg = df_ghg_plot.index.min()
for group in group_order:
    print(f"  {group}: {df_ghg_plot.loc[min_yll_ghg, group]:.3f} GtCO2eq")

In [ ]:
# Manual label positions for GHG
GHG_LABEL_X_POSITIONS = {
    "red_meat": 3,
    "dairy": 30,
    "grain": 7,
    "oil": 10,
    "starchy_vegetable": 5,
    "legumes": 3000,
    "sugar": 5,
    "nuts_seeds": 30000,
    "whole_grains": 500,
    "fruits_vegetables": 3000,
    "eggs_poultry": 30,
}

In [ ]:
fig, ax = plt.subplots(figsize=(3.54, 2.5))  # 90mm x ~63mm
plot_stacked_sensitivity(
    df_ghg_plot,
    colors,
    ax,
    xlabel=YLL_XLABEL,
    ylabel="GHG emissions [GtCO2eq]",
    panel_label="b",
    x_ticks=YLL_XTICKS,
    x_ticklabels=YLL_XTICKLABELS,
    label_x_positions=GHG_LABEL_X_POSITIONS,
    min_height_for_label=0.08,
)
plt.tight_layout()
plt.savefig(
    RESULTS_DIR / "plots" / "yll_ghg_by_food_group.pdf", dpi=300, bbox_inches="tight"
)
plt.show()

## Combined Multipanel Figure

Create a 2x2 figure combining all three plots with panel d reserved for future use.

In [ ]:
# Create 2x2 multipanel figure
fig, axes = plt.subplots(
    2, 2, figsize=(7.08, 5.0)
)  # ~180mm x ~127mm (two-column width)

# Plot a: Food consumption (top-left)
plot_stacked_sensitivity(
    df_plot,
    colors,
    axes[0, 0],
    xlabel=YLL_XLABEL,
    ylabel="Food consumption [kcal/person/day]",
    panel_label="a",
    x_ticks=YLL_XTICKS,
    x_ticklabels=YLL_XTICKLABELS,
    label_x_positions=LABEL_X_POSITIONS,
    y_max=2400,
)

# Plot b: GHG emissions (top-right)
plot_stacked_sensitivity(
    df_ghg_plot,
    colors,
    axes[0, 1],
    xlabel=YLL_XLABEL,
    ylabel="GHG emissions [GtCO2eq]",
    panel_label="b",
    x_ticks=YLL_XTICKS,
    x_ticklabels=YLL_XTICKLABELS,
    label_x_positions=GHG_LABEL_X_POSITIONS,
    min_height_for_label=0.08,
)

# Plot c: Objective breakdown (bottom-left)
plot_objective_sensitivity(
    df_obj,
    axes[1, 0],
    xlabel=YLL_XLABEL,
    panel_label="c",
    x_ticks=YLL_XTICKS,
    x_ticklabels=YLL_XTICKLABELS,
    health_value=CONSTANT_HEALTH_VALUE_PER_YLL,
    ghg_price=CONSTANT_GHG_PRICE,
    label_x_positions=OBJ_LABEL_X_POSITIONS,
    highlight_cat="GHG cost",
)

# Panel d: Empty for now (bottom-right)
axes[1, 1].axis("off")
axes[1, 1].text(
    0.5,
    0.5,
    "d",
    transform=axes[1, 1].transAxes,
    fontsize=9,
    fontweight="bold",
    va="center",
    ha="center",
    alpha=0.3,
)

plt.tight_layout()
plt.savefig(
    RESULTS_DIR / "plots" / "yll_sensitivity_combined.pdf", dpi=300, bbox_inches="tight"
)
plt.show()